# Split and Merge test

## Prepare the environment

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import arviz as az
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import adjusted_rand_score

%matplotlib inline

from bayesmixpy import run_mcmc

In [ ]:
from bayesmixpy import build_bayesmix

In [ ]:
build_bayesmix(4)

## Galaxy and faithful datasets
We test Split and Merge on two real datasets, galaxy (univariate) and faithful (bivariate).
The results are plotted for visual inspection, then they are compared with the ones of Neal2, Neal3, and Neal8 by using Adjusted Rand Index. 

We initialize the priors that are in common for both tests:

In [ ]:
mixing_params=dict()
mixing_params["DP"] = """
gamma_prior {
  totalmass_prior {
    shape: 4.0
    rate: 2.0
  }
}
"""

mixing_params["PY"] = """
fixed_values {
    strength: 1.0
    discount: 0.1
}
"""

algo_params = dict()
for algo in ["Neal2", "Neal3", "Neal8", "SplitMerge"]:
    algo_params[algo] = f"""
algo_id: "{algo}"
rng_seed: 998776
iterations: 5000
burnin: 1000
init_num_clusters: 3
neal8_n_aux: 3
splitmerge_n_restr_gs_updates: 5
splitmerge_n_mh_updates: 1
splitmerge_n_full_gs_updates: 1
"""

### Galaxy

In [ ]:
data = np.loadtxt("../../resources/datasets/galaxy.csv", delimiter=",")
grid = np.loadtxt("../../resources/datasets/galaxy_grid.csv", delimiter=",")

hierarchy_params = """
ngg_prior {
  mean_prior {
    mean: 25.0
    var: 4.0
  }
  var_scaling_prior {
    shape: 0.4
    rate: 0.2
  }
  shape: 4.0
  scale_prior {
    shape: 4.0
    rate: 2.0
  }
}
"""

In [ ]:
%%capture
eval_dens_split_merge=dict()
num_clust_split_merge=dict()
best_clust=dict()
for mixing in ["DP", "PY"]:
    best_clust[mixing] = dict()
    for algo in ["Neal2", "Neal3", "Neal8"]:
        _, _, _, best_clust[mixing][algo], _ = run_mcmc(
            "NNIG", mixing, data, hierarchy_params, mixing_params[mixing],
            algo_params[algo], grid, return_clusters=False, 
            return_num_clusters=False,
            return_best_clus=True)
        
    eval_dens_split_merge[mixing], num_clust_split_merge[mixing], _, \
    best_clust[mixing]["SplitMerge", _], _ = run_mcmc(
        "NNIG", mixing, data, hierarchy_params, mixing_params[mixing],
        algo_params["SplitMerge"], grid, return_clusters=False, 
        return_num_clusters=True,
        return_best_clus=True)

In [ ]:
# Plot best clustering for Split and Merge 
for mixing in ["DP", "PY"]:
    plt.figure(figsize=(12,6), dpi=300)
    plt.title(f"galaxy dataset with {mixing}")
    plt.scatter(data, np.repeat("Split and Merge", len(data)), 
                c=best_clust[mixing]["SplitMerge"], cmap="Set1")

In [ ]:
# Plot density estimation for Split and Merge
for mixing in ["DP", "PY"]:
    fig = plt.figure(figsize=(12,6), dpi=300)
    dens = np.exp(np.mean(eval_dens_split_merge[mixing][0::2], axis=0))
    plt.hist(data, density=True, color='lightgray')
    plt.title(f"galaxy density with {mixing}")
    plt.plot(grid, dens)

In [ ]:
# Compute Adjusted Rand Index between the best clustering of Split and Merge
# and the best clustering of the other algorithms.
print("ADJUSTED RAND INDEX")
for mixing_split_merge in ["DP", "PY"]:
    for mixing in ["DP", "PY"]:
        for algo in ["Neal2", "Neal3", "Neal8"]:
            best_clust_spit_merge = best_clust[mixing_split_merge]['SplitMerge']
            best_clust_competitor = best_clust[mixing][algo]
            print(f"{algo} with {mixing} vs. "+
                f"Split and Merge with {mixing_split_merge}: "+ 
                f"{adjusted_rand_score(best_clust_competitor,best_clust_spit_merge): 0.5f}")
            
best_clust_spit_merge_DP = best_clust["DP"]['SplitMerge']
best_clust_spit_merge_PY = best_clust["PY"]['SplitMerge']
print(f"Split and Merge with DP vs. "+
    f"Split and Merge with PY: "+ 
    f"{adjusted_rand_score(best_clust_spit_merge_DP,best_clust_spit_merge_PY): 0.5f}")

In [ ]:
for mixing in ["DP", "PY"]:
    print(f"ESS with {mixing}: {az.ess(num_clust_split_merge[mixing])}")

### Faithful

In [ ]:
data = np.loadtxt("../../resources/datasets/faithful.csv", delimiter=",")
grid = np.loadtxt("../../resources/datasets/faithful_grid.csv", delimiter=",")

hierarchy_params = """
ngiw_prior {
  mean_prior {
    mean {
      size: 2
      data: 3.0
      data: 3.0
    }
    var {
      rows: 2
      cols: 2
      data: 0.25
      data: 0.0
      data: 0.0
      data: 0.25
    }
  }
  var_scaling_prior {
    shape: 0.4
    rate: 0.2
  }
  deg_free: 4.0
  scale_prior {
    deg_free: 4.0
    scale {
      rows: 2
      cols: 2
      data: 4.0
      data: 0.0
      data: 0.0
      data: 4.0
    }
  }
}
"""

In [ ]:
%%capture
eval_dens_split_merge=dict()
num_clust_split_merge=dict()
best_clust=dict()
for mixing in ["DP", "PY"]:
    best_clust[mixing] = dict()
    for algo in ["Neal2", "Neal3", "Neal8"]:
        _, _, _, best_clust[mixing][algo], _ = run_mcmc(
            "NNW", mixing, data, hierarchy_params, mixing_params[mixing],
            algo_params[algo], grid, return_clusters=False, 
            return_num_clusters=False,
            return_best_clus=True)
        
    eval_dens_split_merge[mixing], num_clust_split_merge[mixing], _, \
    best_clust[mixing]["SplitMerge"], _ = run_mcmc(
        "NNW", mixing, data, hierarchy_params, mixing_params[mixing],
        algo_params["SplitMerge"], grid, return_clusters=False, 
        return_num_clusters=True,
        return_best_clus=True)

In [ ]:
# Plot best clustering for Split and Merge 
for mixing in ["DP", "PY"]:
    plt.figure(figsize=(12,6), dpi=300)
    plt.title(f"faithful dataset with {mixing}")
    plt.scatter(data[:,0],data[:,1], c=best_clust[mixing]["SplitMerge"],
                cmap="Set1")

In [ ]:
# Plot density estimation for Split and Merge
for mixing in ["DP", "PY"]:
    fig = plt.figure(figsize=(12,6), dpi=300)
    dens = np.mean(eval_dens_split_merge[mixing][0::2], axis=0).reshape(-1, 1)
    plot_data = pd.DataFrame(np.hstack([grid, dens]), columns=["x", "y", "z"])
    Z = plot_data.pivot_table(index="x", columns="y", values="z").T.values
    X_unique = np.sort(plot_data.x.unique())
    Y_unique = np.sort(plot_data.y.unique())
    X, Y = np.meshgrid(X_unique, Y_unique)
    plt.contour(X,Y,Z)
    plt.title(f"faithful log-densities with {mixing}")
    #plt.plot(grid, dens)

In [ ]:
# Compute Adjusted Rand Index between the best clustering of Split and Merge
# and the best clustering of the other algorithms.
print("ADJUSTED RAND INDEX")
for mixing_split_merge in ["DP", "PY"]:
    for mixing in ["DP", "PY"]:
        for algo in ["Neal2", "Neal3", "Neal8"]:
            best_clust_spit_merge = best_clust[mixing_split_merge]['SplitMerge']
            best_clust_competitor = best_clust[mixing][algo]
            print(f"{algo} with {mixing} vs. "+
                f"Split and Merge with {mixing_split_merge}: "+ 
                f"{adjusted_rand_score(best_clust_competitor,best_clust_spit_merge): 0.5f}")

# Compute Adjusted Rand Index between Split and Merge with DP and PY mixings.
best_clust_spit_merge_DP = best_clust["DP"]['SplitMerge']
best_clust_spit_merge_PY = best_clust["PY"]['SplitMerge']
print(f"Split and Merge with DP vs. "+
    f"Split and Merge with PY: "+ 
    f"{adjusted_rand_score(best_clust_spit_merge_DP,best_clust_spit_merge_PY): 0.5f}")

In [ ]:
for mixing in ["DP", "PY"]:
    print(f"ESS with {mixing}: {az.ess(num_clust_split_merge[mixing])}")

## High dimensional synthetic datasets
We test Split and Merge and compare it with Neal3 to check how they work in high dimensionality. Change the value of variable `d` to set the dimensionality.

In [ ]:
def compute_hierarchy_params(d):
    hierarchy_params = """
fixed_values {
  mean {
    size:"""+str(d)+"\n"+ ("data:0.0\n")*d +"""}
  var_scaling: 1.0
  deg_free: """+str(d)+"""
  scale {
    rows: """+str(d)+"""
    cols: """+str(d)+"""
"""

    for i in range(d):
        for j in range(d):
            hierarchy_params=hierarchy_params+"data: "+str(float(i==j))+\
            "\n"
    hierarchy_params=hierarchy_params+"}}"
    
    return hierarchy_params

In [ ]:
n_data = 50
seed=45245

In [ ]:
mixing_params=dict()
mixing_params["DP"] = """
gamma_prior {
  totalmass_prior {
    shape: 4.0
    rate: 2.0
  }
}
"""

mixing_params["PY"] = """
fixed_values {
    strength: 1.0
    discount: 0.1
}
"""

algo_params = dict()
for algo in ["Neal3", "SplitMerge"]:
    algo_params[algo] = f"""
algo_id: "{algo}"
rng_seed: 998776
iterations: 15000
burnin: 100
init_num_clusters: 1
neal8_n_aux: 3
splitmerge_n_restr_gs_updates: 10
splitmerge_n_mh_updates: 1
splitmerge_n_full_gs_updates: 2
"""

In [ ]:
%%capture
d=6

hierarchy_params = compute_hierarchy_params(d)

data = np.zeros((n_data, d))
n_data_first_clust = round(n_data/2)
n_data_second_clust = n_data-n_data_first_clust

correct_clustering = np.zeros(data.shape[0])
correct_clustering[0:n_data_first_clust] = 1

data = ss.multivariate_normal(
  np.repeat(10/np.sqrt(d), d), np.identity(d), False, seed).rvs(n_data)

data[0:n_data_first_clust]=-data[0:n_data_first_clust]

# In this case the grid is not important because we are not interested 
# in the density. Therefore, we put it equal to data.
grid = data

num_clust = dict()
best_clust = dict()
for mixing in ["DP","PY"]:
    num_clust[mixing] = dict()
    best_clust[mixing] = dict()
    for algo in ["Neal3", "SplitMerge"]:
        _, num_clust[mixing][algo], _, best_clust[mixing][algo], _ = run_mcmc(
            "NNW", mixing, data, hierarchy_params, mixing_params[mixing],
            algo_params[algo], grid, return_clusters=False, 
            return_num_clusters=True,
            return_best_clus=True)        

In [ ]:
print(f"{d} dimensions")
print("Adjusted Rand Index w.r.t. the correct clustering: ")
for mixing in ["DP","PY"]:
    for algo in ["Neal3", "SplitMerge"]: 
        ari =adjusted_rand_score(best_clust[mixing][algo], correct_clustering)
        print(f"{algo} with {mixing}: {ari: .5f}")

In [ ]:
for mixing in ["DP","PY"]:
    for algo in ["Neal3", "SplitMerge"]:  
        plt.figure(figsize=(20,5), dpi=300)
        plt.title(f"Trace plot of the number of clusters of {algo} with {mixing}, {d} dimensions")
        plt.plot(num_clust[mixing][algo])

Changing the parameter $d$ we can see that, in general, Split and Merge recognizes correctly the two clusters when $d\le 10$ and fails with $d\ge 11$, while Neal3 fails with $d\ge 6$.

One particular case is with with 7 dimensions: Split and Merge moves in a few iterations to 2 clusters and oscillates between 2 and 3, while Neal3 requires 3000 iterations (with DP) or 7000 iterations (with PY) to move to two clusters. Therefore, also in the cases when Neal3 recognizes the two clusters, Split and Merge requires far less iterations!